In [1]:
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
import pandas as pd
import h5py
from os.path import join as pjoin
import csv
import itertools
import argparse
import time
from tqdm import tqdm
import multiprocessing
from nilearn import datasets
from nilearn import surface

### NSD data preprocessing workflow
1. load 3d data from hdf5
2. frame the visual areas
3. start convolution (2d or 3d)

In [2]:
subj = 'subj02'
pic=11

In [3]:
basedir = f'/Volumes/side_project/nsd/{subj}'

In [4]:
# get the voxel index for each subject
fsaverage_betas = nib.load('/Volumes/side_project/nsd/fsaverage/subj01/lh.betas_session01.mgh')
print(fsaverage_betas.shape)

(163842, 1, 1, 750)


In [5]:
## get the 3d coordinates of the fsaverage mesh surface
fsaverage = datasets.fetch_surf_fsaverage(mesh="fsaverage")
left_pial = surface.load_surf_mesh(fsaverage["pial_left"])
right_pial = surface.load_surf_mesh(fsaverage["pial_right"]) 

In [6]:
print(sum(surface.load_surf_mesh(fsaverage["flat_left"]).coordinates[:,2]))

58819.93439013256


In [7]:
# import mask
roidir = '/Users/yilewang/Downloads/'+subj
fmri_dir = pjoin(roidir, 'roi_masks')

# load npy file
lh_all_data = np.load(pjoin(fmri_dir, 'lh.all-vertices_fsaverage_space.npy'))
rh_all_data = np.load(pjoin(fmri_dir, 'rh.all-vertices_fsaverage_space.npy'))

In [8]:
# apply mask to the whole brain voxels

# Map the fMRI data onto the brain surface map
visual_vertex_left = lh_all_data[np.where(lh_all_data)[0]]
visual_vertex_right = rh_all_data[np.where(rh_all_data)[0]]


# get the 3d coordinates of the visual cortex
visual_left_3d = left_pial.coordinates[np.where(visual_vertex_left)[0]]
visual_right_3d = right_pial.coordinates[np.where(visual_vertex_right)[0]]

print(left_pial.coordinates)

[[-38.83455   -19.019356   66.90841  ]
 [-16.554127  -69.25585    60.977936 ]
 [ -9.5535755  -8.218807   46.25186  ]
 ...
 [-34.570942  -24.75986   -26.428991 ]
 [-34.64751   -24.712444  -25.722996 ]
 [-35.078686  -24.75963   -25.221867 ]]


In [9]:
from sklearn import datasets, manifold
import matplotlib.pyplot as plt
from matplotlib import ticker
import mpl_toolkits.mplot3d 
n_samples = 1500
S_points, S_color = datasets.make_s_curve(len(left_pial.coordinates), random_state=0)
print(S_points.shape)
def add_2d_scatter(ax, points, points_color, title=None):
    x, y = points.T
    ax.scatter(x, y, c=points_color, s=50, alpha=0.8)
    ax.set_title(title)
    ax.xaxis.set_major_formatter(ticker.NullFormatter())
    ax.yaxis.set_major_formatter(ticker.NullFormatter())

def plot_2d(points, points_color, title):
    fig, ax = plt.subplots(figsize=(3, 3), facecolor="white", constrained_layout=True)
    fig.suptitle(title, size=16)
    add_2d_scatter(ax, points, points_color)
    plt.show()


n_neighbors = 12
n_components = 2

isomap = manifold.Isomap(n_neighbors=n_neighbors, n_components=n_components, p=1)

S_isomap = isomap.fit_transform(S_points)
print(n_components)
plot_2d(S_isomap, S_color, "Isomap Embedding")

(163842, 3)


: 

### 2D and 3D convolution

In [ ]:
m = nn.Conv3d(16, 33, 3, stride=2)
# non-square kernels and unequal stride and with padding
m = nn.Conv3d(16, 33, (3, 5, 2), stride=(2, 1, 1), padding=(4, 2, 0))
input = torch.randn(20, 16, 10, 50, 100)
output = m(input)